*The code snippet assumes Anaconda 5.2.0 version of Python virtual environment*

<div class="alert alert-info">
    <h4>Acknowledgement</h4>
    <p>The materials on this post are based the two research paper, <a href="https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf">Distributed Representations of Words and Phrases and their Compositionality</a> (Mikolov et al., 2013) and <a href="https://arxiv.org/pdf/1411.2738.pdf">word2vec Parameter Learning Explained</a> (Rong, 2014).</p>
</div>

Word Embedding - represent words in a vector space



## Paradigm Shift in Word Embedding: Frequency-Based to Prediction-Based

The 

## Prediction-based word-embedding: Word2Vec Skip-Gram

picture of mikolov skip-gram diagram + my diagram of window_size = 2

skip-gram predicts context words given a center word.


<div class="alert alert-info">
    <h4>Notes: CBOW and Skip-Gram</h4>
    <p>There are two models for Word2Vec: <i>Continous Bag Of Words (CBOW)</i> and <i>Skip-Gram</i>. While Skip-Gram model predicts context words given a center word, CBOW model predicts a center word given context words. According to Mikolov:</p>
    <p><i>Skip-gram</i>: works well with small amount of the training data, represents well even rare words or phrases</p>
    <p><i>CBOW</i>: several times faster to train than the skip-gram, slightly better accuracy for the frequent words</p>
    <p>Skip-Gram model is a better choice most of the time due to its ability to predict infrequent words, but this comes at the price of increased computational cost. If training time is a big concern, and you have large enough data to overcome the issue of predicting infrequent words, CBOW model may be a more viable choice. The details of CBOW model won't be covered in this post.</p>
</div>

A natural question is, why do we predict context words? We do it so that we can optimize the weight matrix and get the correct word-embedding matrix. This is described [below]:


This will make more sense as you understand the structure of Skip-Gram model. 

<div class="alert alert-info">
    <h4>Notes: Why predict context words?</h4>
    <p>T</p>
</div>

## Derivation of Cost Function

Skip-Gram model seeks to optimize the word embedding matrix by correctly predicting context words, given a center word. In the other words, the model wants to maximize the probability of correctly predicting all context words at the same time, given a center word. Mathematically, it can be expressed as:

<div id="eq-1" style="font-size: 1rem;">$$ max \, p(w_{1}, w_{2}, ... , w_{C}|w_{center}) \tag{1} $$</div>

where $C$ is the window size. Recall that in statistics, the probability of $A$ given $B$ is expressed as $P(A|B)$. Then, natural log is taken on <a href="#eq-1">eq (1)</a> to simplify taking derivatives. 

<div id="eq-2" style="font-size: 1rem;">$$ max \, log \, p(w_{1}, w_{2}, ... , w_{C}|w_{center}) \tag{2} $$</div>

<div class="alert alert-info">
    <h4>Notes: Why take a natural log?</h4>
    <p>In machine learning, it is a common practice to take a natural log to the objective function to simplify taking derivatives. For example, a multinomial regression classifer called Softmax (details explained <a href="#Softmax-Output-Layer-($y_{pred}$)">below</a>) has the following probability function:</p>
    <p><center style="margin-top: 20px">$p(x_i) = \frac{e^{x_i}}{\sum_{j=1}e^{x_{j}}}$</center></p>
    <p>Taking a log simplifies the function:</p>
    <p><center style="margin-top: 20px">$log \, p(x_i) = x_i - log \, {\sum_{j=1}e^{x_{j}}}$</center></p>
    <p>Depending on a model, the argument ($x_i$) passed into the probability function ($p$) can be complicated, and simplifying the original softmax function helps with taking the derivatives in the future.</p> 
    <p>Taking a log does not affect the optimized weights ($\theta$), because natural log is a <u>monotonically increasing</u> function. This means that increasing the value of $x$-axis results in increasing the value of $y$-axis. This is important because it ensures that the maximum value of the original probability function occurs at the same point as the log probability function. Therefore:</p>
    <p><center style="margin-top: 20px">$max \, p(x_i) = max \, log \, p(x_i)$</center></p>
</div>

In Skip-Gram, softmax function is used for context words classfication. The details are explained <a href="#Softmax-Output-Layer-($y_{pred}$)">below</a>. Softmax in Skip-Gram has the following equation:

<div id="eq-3" style="font-size: 1rem;">$$ p(w_{context}|w_{center}) = \frac{exp(W_{output_{(context)}} \cdot h)}{\sum^V_{i=1}exp(W_{output_{(i)}} \cdot h)} \tag{3} $$</div>

$W_{output_{(context)}}$ is a row vector for a context word from the output embedding matrix, and $h$ is the hidden (projection) layer word vector for a center word. Softmax function is then plugged into the <a href="#eq-2">eq (2)</a> to yield a new objective function that maximizes the probability of observing all $C$ context words, given a center word:

<div id="eq-4" style="font-size: 1rem;">$$ max \, log \, \prod_{c=1}^{C} \frac{exp(W_{output_{(c)}} \cdot h)}{\sum^V_{i=1}exp(W_{output_{(i)}} \cdot h)} \tag{4} $$</div>

<div class="alert alert-info">
    <h4>Notes: Probability Product</h4>
    <p>In statistics, probability of observing $C$ multiple events at the same time is computed by the product of each event's probability.</p>
    <p><center style="margin-top: 20px">$$p(x_{1}, x_{2} ... x_{C}) = p(x_{1}) \times p(x_{2}) \, \times \, ... \, \times \, p(x_{C})$$</center></p>
    <p>This can be shortened with a product notation:</p>
    <p><center style="margin-top: 20px">$$p(x_{1}, x_{2} ... x_{C}) = \prod_{c=1}^{C}p(x_{c})$$</center></p>
</div>

However, in machine learning, the convention is to minimize the cost function, not to maximize it. To stick to the convention, we add a negative sign to <a href="#eq-4">eq (4)</a>. This can be done because minimizing a negative log-likelihood is equivalent to maximizing a positive log-likelihood. Therefore, the cost function we want to minimize becomes:

<div id="eq-5" style="font-size: 1rem;">$$ J(\theta; w^{(t)}) = -log \, \prod_{c=1}^{C} \frac{exp(W_{output_{(c)}} \cdot h)}{\sum^V_{i=1}exp(W_{output_{(i)}} \cdot h)} \tag{5} $$</div>

where $c$ is the index of the context word around the center word ($w_{t}$). $t$ is the index of the center word within a corpus of size $T$. Taking a log to the softmax function allows us to simplify the expression into simpler forms because we can split the fraction into addtion of the numerator and the denominator:

<div id="eq-6" style="font-size: 1rem;">
$$
J(\theta; w^{(t)}) = - \sum_{c=1}^{C}(W_{output_{(c)}} \cdot h) + C \cdot log \sum^V_{i=1}exp(W_{output_{(i)}} \cdot h) \tag{6}
$$
</div>

Different paper uses different notations for the cost function. To stick to the notation used in the <a href="https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf">Word2Vec original paper</a>, some of the notations in <a href="#eq-6">eq (6)</a> can be changed. However, they are all equivalent:

<div id="eq-7" style="font-size: 1rem;">$$J_{t}(\theta;w^{(t)}) = -\sum_{-m\leq j \leq m} \log p(w_{t+j} \mid w_t ; \theta) \tag{7}$$</div>

It should be noted that both <a href="#eq-6">eq (6)</a> and <a href="#eq-7">eq (7)</a> assumes stochastic gradient descent, which means that for each training sample in the corpus ($w^{(t)}$), one update is made to the weight matrix ($\theta$). The cost function expressed in the <a href="https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf">Word2Vec original paper</a> shows batch gradient descent, which means that one update was made for all $T$ training samples:

<div id="eq-8" style="font-size: 1rem;">$$J(\theta) = -\frac{1}{T} \sum^T_{t=1} \sum_{-m\leq j \leq m} \log p(w_{t+j} \mid w_t ; \theta) \tag{8}$$</div>

However in Word2Vec, batch gradient descent is almost never used due to its high computational cost. The author of the paper stated that he used stochastic gradient descent for training. Read the below <a href="#stochastic">notes</a> for more information.

### Window Size of Skip-Gram

Softmax regression (or multinomial logistic regression) is a generalization of logistic regression to the case where we want to handle multiple classes. A general form of the softmax regression looks like this:

<div id="eq-9" style="font-size: 1rem;">
$$J(\theta) = 
-\frac{1}{T} 
\sum^T_{t=1}
\sum^K_{k=1}
log 
\frac
{exp(\theta^{(k)\top}x^{(t)})}
{\sum^K_{j=1}
exp(\theta^{(j)\top}x^{(t)})} \tag{10}$$</div>

where $T$ is the number of training samples, and $K$ is the number of labels to classify. In NLP applications, $K = V$, because there are $V$ unique vocabulary we need to classify in a vector space. $V$ can easily exceed tens of thousands. Skip-Gram tweaks this a little, and replaces $K$ with a variable called **window size**. Window size is a hyper parameter of the model with a typical range of $[1, 10]$. Recall that Skip-Gram is a model that attempts to predict neighboring words of a center word. The farther the distance away from the current word, the less information it carries. Therefore, it doesn't have to predict all $V$ vocab in the corpus that may be 100 words away from it, but instead predict only a few, 1~10 neighboring context words.

# Neural Network Structure of Skip-Gram

How is neural network used to minimize the cost functoin described in <a href="#eq-9">eq (9)</a>? One needs to look into the structure of the Skip-Gram model to gain insights about their correlation. 

For illustration purpose, let's assume that the entire corpus is composed of the quote from the Game of Thrones, <i>"The man who passes the sentence should swing the sword"</i>, by Ned Stark. There are 10 words ($T = 10$), and 8 unique words ($V = 8$). 

Note that in real life, the corpus is much bigger than just one sentence. 

<blockquote class="quote">
  The man who passes the sentence should swing the sword.
  <span>- Ned Stark</span>
</blockquote>

We will use <code>window=1</code>, and assume that <i>'passes'</i> is the current center word, making <i>'who'</i> and <i>'the'</i> context words. <code>window</code> is a hyper-parameter that can be empirically tuned. It typically has a range of 2-10.

<div class="row give-margin-inline-big-plot">
    <div class="col"><img src="jupyter_images/quote_ned_stark.png"></div>
    <div class="col-12"><p class="image-description">Figure 1: Training Window</p></div>
</div>

For illustration purpose, a three-dimensional neural net will be constructed. In *gensim*, this can be implemented by setting <code>size=3</code>. This makes $N = 3$. Just like <code>window</code>, <code>size</code> is a hyper-parameter that can be empirically tuned. In real life, a typical Word2Vec model has 200-600 neurons. 

<pre>
    <code class="language-python">
        from gensim.models import Word2Vec

        model = Word2Vec(corpus, size=3, window=1)
    </code>
</pre>

This means that the input weight matrix ($W_{input}$) will have a size of $8 \times 3$, and output weight matrix ($W_{output}^T$) will have a size of $3 \times 8$. Recall that the corpus, <i>"The man who passes the sentence should swing the sword"</i>, has 8 unique vocabularies ($V = 8$). 

<div class="row">
    <div class="col"><img src="jupyter_images/word2vec_skip-gram.png" style="margin: 0;"></div>
    <div class="col-12"><p class="image-description">Figure 2: Skip-Gram model structure</p></div>
</div>

## Training: Forward Propagation

The word embedding matrices ($W_{input}$, $W_{output}$) in Skip-Gram are optimized through forward and backward propagations. For each iteration of forward + backward propagations, the model learns to reduce prediction error, thus achieving higher quality embedding matrices that better captures relationships among words. 

Forward propagation includes obtaining the probability distribution of words ($y_{pred}$ in *figure 2*) given a center word, and backward propagation includes calculating the prediction error, and updating the weight (embedding) matrices to minimize the prediction error.

## Derive equations for Y_predict

### Input Layer ($x$)

The input layer is a $V$-dim one-hot encoded vector. Every element in the vector is 0 except one element that corresponds to the center word. Input vector is multiplied with the input weight matrix ($W_{input}$) of size $V \times N$, and yields a hidden (projection) layer ($h$) of $N$-dim vector. Because the input layer is one-hot encoded, it makes the input weight matrix ($W_{input}$) to behave like a *look-up table* for the center word. Assuming epoch number of 1 (<code>iter=1</code> in *gensim* Word2Vec implementation) and stochastic gradient descent, the input vector is injected into the network $T$ times for every word in the corpus and makes $T$ updates to the weight matrix ($\theta$).

<div class="row give-margin-inline-big-plot">
    <div class="col"><img src="jupyter_images/one-hot-vector.png"></div>
    <div class="col-12"><p class="image-description">Figure 2: One-hot encoded input vector and parameter update</p></div>
</div>

<div class="alert alert-info">
    <h4>Notes: Stochastic Gradient Descent</h4>
    <p><a id="stochastic"></a>The goal of any machine learning model is to find the optimal values of a weight matrix ($\theta$) to minimize prediction error. A general update equation for weight matrix looks like the following:</p>
    <p><center style="margin-top: 20px">$\theta_{new}=\theta_{old}-\eta\cdot\nabla_{\theta}J(\theta)$</center></p>
    <p>$\eta$ is learning rate, $\nabla_{\theta}$ is gradient for the weight matrix, and $J(\theta)$ is the cost function that has different forms for each model. The cost function for the Skip-Gram model proposed in the <a href="https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf">Word2Vec original paper</a> has the following equation:</p>
    <p><center style="margin-top: 20px">$$J(\theta) = -\frac{1}{T} \sum^T_{t=1} \sum_{-m\leq j \leq m} \log p(w_{t+j} \mid w_t ; \theta)$$</center></p>
    <p>Here, what gives us headache is the expression, $\frac{1}{T} \sum^T_{t=1}$, because $T$ can be larger than billions or more in many NLP applications. It is basically telling us that billions of iterations need to be computed to make just one update to the weight matrix ($\theta$). In order to mitigate this computational burden, the author of the paper states that Stochastic Gradient Descent (SGD) was used for parameter optimization. SGD removes the expression, $\frac{1}{T} \sum^T_{t=1}$, from the cost function and performs parameter update for each training example, $w^{(t)}$:</p>
    <p><center style="margin-top: 20px">$$J_{t}(\theta;w^{(t)}) = -\sum_{-m\leq j \leq m} \log p(w_{t+j} \mid w_t ; \theta)$$</center></p>
    <p>Then, the new parameter update equation for SGD becomes:</p>
    <p><center style="margin-top: 20px">$\theta_{new}=\theta_{old}-\eta\cdot\nabla_{\theta}J_{t}(\theta;w^{(t)})$</center></p>
    <p>The original vanilla graident descent makes $1$ parameter update for $T$ training samples, but the new update equation using SGD makes $T$ parameter update for $T$ training samples. However, this comes at the price of higher fluctuation (or variance) in minimizing prediction error.</p>
</div>

#### Input and Output Weight Matrix  ($W_{input}$, $W_{output}$)

Why does Skip-Gram model attempt to predict context words given a center word? How does predicting context words help with quantifying words and representing them in a vector space? In fact, the ultimate goal of the model is not to predict context words, but to construct the word embedding matrices ($W_{input}$, $W_{output}$) that best caputure relationship among words in a vector space. Skip-Gram achieves this by using a neural net — it optimizes the weight (embedding) matrices by adjusting the weight matrix to minimize the prediction error ($y_{pred} - y_{true}$). This will make more sense once you understand how the embedding matrix behaves like a *look-up table*.

Each row in a word-embedding matrix is a word-vector for each word. Consider the following word-embedding matrix, $W_{input}$.

<div class="row give-margin-inline-plot">
    <div class="col"><img src="jupyter_images/embedding_matrix_input.png" style="height: 300px;"></div>
    <div class="col-12"><p class="image-description">Figure 3: Word-embedding matrix, $W_{input}$</p></div>
</div>

The words of our interest are *"passes"* and *"swing"*. *"passes"* has a word vector of $[0.1 \quad 0.2 \quad 0.7]$ and *"swing"* has $[-2 \quad 0.2 \quad 0.8]$. Since we set the size of the weight matrix to be <code>size=3</code>, the matrix is three-dimensional, and can be visualized in a 3D vector space:

<div class="row give-margin full_screen_margin">
    <div class="col"><img src="jupyter_images/word2vec_3d.png" style="height: 300px;"></div>
    <div class="col-12"><p class="image-description">Figure 4: 3D visualization of word vectors in embedding matrix</p></div>
</div>

Optimizing the embedding (weight) matrices means placing words in a correct vector space, and as a result, the model will be able to capture meaningful relationships among words in the vector space.

<div class="alert alert-info">
    <h4>Notes: $\theta$ in cost function</h4>
    <p>There are two weight matrices that need to be optimized in Skip-Gram model: $W_{input}$ and $W_{output}$. Often times in neural net, the weights are expressed as $\theta$. In Skip-Gram, $\theta$ is a concatenation of input and output weight matrices — $[W_{input} \quad W_{output}]$.</p>
    <div id="eq-2-5">$$ \theta = [W_{input} \quad W_{output}] = \left[ \begin{array}{l} u_{the} \\ u_{passes} \\ \vdots \\ u_{who} \\  v_{the} \\ v_{passes} \\ \vdots \\ v_{who} \end{array} \right] \in \mathbb{R}^{2NV} \tag{2.5}$$</div>
    <p>$\theta$ has a size of $2V \times N$, where $V$ is the number of unique vocab in a corpus, and $N$ is the number of neurons in the network. $2$ is multipled to $V$ because there are two weight matrices, $W_{input}$ and $W_{output}$. $u$ is a word vector from $W_{input}$ and $v$ is a word vector from $W_{output}$. The each word vectors are $N$-dim row vectors from input and output embedding matrix.</p>
</div>

#### Hidden (Projection) Layer ($h$)

Skip-Gram uses a neural net with one hidden layer. In the context of natural language processing, hidden layer is often referred to as a *projection* layer, because $h$ is essentially an 1D vector projected by the one-hot encoded input vector.

<div class="row give-margin full_screen_margin">
    <div class="col"><img src="jupyter_images/skip-gram_lookup.png"></div>
    <div class="col-12"><p class="image-description">Figure 5: Computing projection layer</p></div>
</div>

$h$ is obtained by multiplying the input word embedding matrix with the $V$-dim input vector:

<div style="font-size: 1rem;">$$h = W_{input}^T \cdot x \tag{2.5}$$</div>

#### Softmax Output Layer ($y_{pred}$)

The output layer is a $V$-dim probability distribution of all unique words in the corpus, given a center word. In statistics, the conditional probability of $A$ given $B$ is denoted as $p(A|B)$. In Skip-Gram, we use the notation, $p(w_{context}| w_{center})$, to denote the conditional probability of a context word given a center word. In Skip-Gram, it is obtained by using the softmax function,

<div style="font-size: 1rem;">$$ p(w_{context}|w_{center}) = \frac{exp(W_{output_{(context)}} \cdot h)}{\sum^V_{i=1}exp(W_{output_{(i)}} \cdot h)} \in \mathbb{R}^{1} \tag{2.5} $$</div>

where $W_{output_{(i)}}$ is the $i$-th row vector of size $1 \times N$ from the output embedding matrix, $W_{output_{context}}$ is also a row vector of size $1 \times N$ from the output embedding matrix corresponding to the context word, $V$ is the size of unique vocab in the corpus, and $h$ is the hidden (projection) layer of size ($N \times 1$). The output is an $1 \times 1$ scalar value of probability of range $[0, 1]$.

This probability is computed $V$ times to obtain probability distribution of all unique words in the corpus, given a center word. 

<div style="font-size: 1rem;">$$ \left[ \begin{array}{c} p(w_{1}|w_{center}) \\ p(w_{2}|w_{center}) \\ p(w_{3}|w_{center}) \\ \vdots \\ p(w_{V}|w_{center}) \end{array} \right] = \frac{exp(W_{output} \cdot h)}{\sum^V_{i=1}exp(W_{output_{(i)}} \cdot h)} \in \mathbb{R}^{V}\tag{2.5} $$</div>

$W_{output}$ in the denominator of eq 3333 has size $V \times N$. Multiplying $W_{output}$ with $h$ of size $N \times 1$ will yield a dot product vector of size $V \times 1$. This dot product vector goes through the softmax function:

<div class="row give-margin full_screen_margin">
    <div class="col"><img src="jupyter_images/softmax_function.png" style="height: 300px;"></div>
    <div class="col-12"><p class="image-description">Figure 6: softmax function transformation</p></div>
</div>

The exponentiation ensures that the transformed values are positive, and the normalization factor in the denominator ensures that the values have a range of $[0, 1]$. The result is a probability distribution of all words in the corpus, given a center word.

<div class="alert alert-info">
    <h4>Notes: Negative Sampling</h4>
    <p>Softmax function in Skip-Gram has the following equation:</p>
    <p><div style="font-size: 1rem; margin-top: 20px;">$$ p(w_{context}|w_{center}) = \frac{exp(W_{output_{(context)}} \cdot h)}{\sum^V_{i=1}exp(W_{output_{(i)}} \cdot h)}$$</div></p>
    <p>The normalization factor in the denominator requires special attention because the very large size of $V$ makes the computation very expensive. The author of the <a href="https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf">Word2Vec original paper</a> came up with a technique called <i>Negative Sampling</i> to avoid this computational burden. It replaces the expression, $\sum^V_{i=1}$, to $\sum^k_{i=1}$, where $k$ is the number of negative samples. Then, the new softmax function is,</p>
    <p><div style="font-size: 1rem; margin-top: 20px;">$$ p(w_{context}|w_{center}) = \frac{exp(W_{output_{(context)}} \cdot h)}{\sum^k_{i^{'}=1}exp(W_{output_{(i^{'})}} \cdot h)}$$</div></p>
    <p>where $k$ is usually chosen between 5-20. $k=20$ is used for small samples, and $k=5$ is used for big samples. $V$ can be more then tens of thousands or even millions, but $k$ is much smaller than $V$. The very small size of $k$ significantly reduces the computational cost of the normalization factor. In <i>gensim</i>, negative sampling is applied by default with <code>Word2Vec(negative=5, ns_exponent=0.75)</code>, where <code>negative</code> is the number of $k$-negative samples, and <code>ns_exponent</code> is a hyperparameter related to negative sampling, of range $(0, 1)$. The details of the methodology behind negative sampling deserves another fully devoted article, and as such, it won't be covered here.</p>
</div>

## Training: Backward Propagation

Once the predictions were made during the forward pass, the model can now calculate the prediction error and learn to optimize the weight matrices ($W_{input}$, $W_{output}$) in a way that minimizes the error.

### Derive parameter update equations

#### Prediction Error ($y_{pred} - y_{true}$)







<div class="alert alert-info">
    <h4>Notes: Window Size</h4>
    <p>Algorithm efficiency optimization with window size</p>
    <p></p>
</div>


$$J(\theta) = -\frac{1}{T} \sum^T_{t=1} \sum_{-m\leq j \leq m} \log p(w_{t+j} \mid w_t ; \theta)$$

Skip-Gram model predicts context words - why? -> because its a look up table.

Lookup table illustration

Representing in Equations

3D vectorization on 3D plot.

FAQ


Error 3D plot at the introduction of minimizing J(theta) cost function.

If the center word is the same as context word, it is skipped. https://github.com/RaRe-Technologies/gensim/blob/f267abf94e84484047fb7569ebacba5626bc8391/gensim/models/word2vec.py#L129

## Numerical Demonstration

In [ ]:
len(context) * np.log(np.sum(np.exp(u))) - np.sum([u[label == 1] for label in context])

Hierarchical softmax, where the words are arranged in a tree (similar to a decision tree), making the complexity logarithmic.


Negative sampling, Where the system learns to distinguish the correct answer from a sample of a few incorrect answers.